In [1]:
import time
import numpy as np
import sys
from ucimlrepo import fetch_ucirepo

sys.path.insert(0, "./Optimization-lib")
from gradient_descent import GradientDecent
from stochastic_gradient_descent import StochGradientDecent
from function_wrapper import FunctionWrapper
from lrs import exponential_decay, gradient
from output import pretty_dataset_print
from all_stats import measure_resources

wine_quality = fetch_ucirepo(id=186)
X = wine_quality.data.features.values
y = wine_quality.data.targets.values.ravel()

def normalize(X_train):
    mean = X_train.mean(axis=0)
    std = X_train.std(axis=0)
    std[std == 0] = 1
    return (X_train - mean) / std

X = normalize(X)

def generate_weight_bounds(X, abs_bound):
    return [[-abs_bound, abs_bound] for _ in range(X.shape[1] + 1)]

def generate_start(X):
    return [1 for _ in range(X.shape[1] + 1)]

bounds = generate_weight_bounds(X, 1000)
start = generate_start(X)


sgd = StochGradientDecent(
    exponential_decay(0.01, 0.0001),
    bounds,
    X, y,
    batch_size=5
)
error_min_sgd = sgd.find_min(start, 1000000)
time_sgd, mem_sgd, cpu_sgd = measure_resources(sgd.find_min, start, 1000000)
print(f"SGD результаты:")
print(f"Время: {time_sgd:.2f} сек")
print(f"Пиковое использование памяти: {mem_sgd:.2f} MiB")
print(f"Использование CPU: {cpu_sgd:.2f}%\n")


def create_mse_function(X, y):
    def mse(weights):
        predictions = weights[0] + X.dot(weights[1:])
        return np.mean((y - predictions) ** 2)
    return FunctionWrapper(mse)
mse_func = create_mse_function(X, y)
gd = GradientDecent(
    exponential_decay(0.01, 0.0001),
    mse_func,
    bounds,
    0.0001
)
error_min_gd = gd.find_min(start, 1000)
time_gd, mem_gd, cpu_gd = measure_resources(gd.find_min, start, 1000)
print(f"GD результаты:")
print(f"Время: {time_gd:.2f} сек")
print(f"Пиковое использование памяти: {mem_gd:.2f} MiB")
print(f"Использование CPU: {cpu_gd:.2f}%\n")



SGD результаты:
Время: 364.07 сек
Пиковое использование памяти: 45.92 MiB
Использование CPU: 97.10%



KeyboardInterrupt: 